***GENERATED CODE FOR scoringdatadaily PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run scoringdatadailyHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5fd8a3004a8cbd1858166466/part-00000-tid-2798526237088427705-b85824c1-124d-4ab4-be18-abfba9a6bb11-54290-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "Select DISTINCT cd.*, cdt.tx_count from \r\n ( \r\n select DISTINCT t.ChargeDetailID as cid, count(*) as tx_count from transactions t \r\n where t.CloseDate is  NULL and t.DateOfEntry= DATEADD(DAY, -1, Convert(date, getdate()))  and t.Amount > .01\r\n group by t.ChargeDetailID\r\n ) as cdt \r\n inner join \r\n (\r\n SELECT DISTINCT cd.ChargeDetailID,\r\n cd.LoadDate,\r\n Month(cd.DateOfService) AS MonthOfEntry ,\r\n Day(cd.DateOfService) AS DayOfEntry ,\r\n cd.PatientID, cptc.CPTCode,\r\n dc.DiagnosisCode as pdiagnosis1,\r\n dc2.DiagnosisCode as pdiagnosis2,\r\n FLOOR(DATEDIFF(DAY,  p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n p.Gender,\r\n d.DepartmentCategory, \r\n s.SpecialityCategory,\r\n predetermination_ind=(SELECT Case WHEN (apa.predetermination_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n med_necessity_req_ind=(SELECT Case WHEN (apa.med_necessity_req_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n is_authorized=(SELECT Case WHEN (apa.auth_code IS NOT NULL) THEN 1 ELSE 0 END),\r\n label=(SELECT CASE tr.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\n from ChargeDetail cd \r\n inner join TransactionReason r on r.ChargeDetailID = cd.ChargeDetailID \r\n inner join TransactionReasonCode tr on tr.TransactionReasonCodeID = r.TransactionReasonCodeID\r\n inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n left outer join AuthorizationsPA apa WITH (NOLOCK) on cd.PatientID = apa.person_id and apa.payer_id = cd.PayerID and apa.end_date >= cd.DateOfService\r\n ) as cd\r\n on cdt.cid = cd.ChargeDetailID", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run scoringdatadailyHooks.ipynb
try:
	#sinkPreExecutionHook()

	dbfssink = DBFSConnector.put(df, "{'url': '/ScoringFiles/DenialsScenario/score020.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'source_attributes': {'url': '/FileStore/platform/sampleData/5fd8a3004a8cbd1858166466/part-00000-tid-2798526237088427705-b85824c1-124d-4ab4-be18-abfba9a6bb11-54290-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "Select DISTINCT cd.*, cdt.tx_count from \r\n ( \r\n select DISTINCT t.ChargeDetailID as cid, count(*) as tx_count from transactions t \r\n where t.CloseDate is  NULL and t.DateOfEntry= DATEADD(DAY, -1, Convert(date, getdate()))  and t.Amount > .01\r\n group by t.ChargeDetailID\r\n ) as cdt \r\n inner join \r\n (\r\n SELECT DISTINCT cd.ChargeDetailID,\r\n cd.LoadDate,\r\n Month(cd.DateOfService) AS MonthOfEntry ,\r\n Day(cd.DateOfService) AS DayOfEntry ,\r\n cd.PatientID, cptc.CPTCode,\r\n dc.DiagnosisCode as pdiagnosis1,\r\n dc2.DiagnosisCode as pdiagnosis2,\r\n FLOOR(DATEDIFF(DAY,  p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n p.Gender,\r\n d.DepartmentCategory, \r\n s.SpecialityCategory,\r\n predetermination_ind=(SELECT Case WHEN (apa.predetermination_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n med_necessity_req_ind=(SELECT Case WHEN (apa.med_necessity_req_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n is_authorized=(SELECT Case WHEN (apa.auth_code IS NOT NULL) THEN 1 ELSE 0 END),\r\n label=(SELECT CASE tr.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\n from ChargeDetail cd \r\n inner join TransactionReason r on r.ChargeDetailID = cd.ChargeDetailID \r\n inner join TransactionReasonCode tr on tr.TransactionReasonCodeID = r.TransactionReasonCodeID\r\n inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n left outer join AuthorizationsPA apa WITH (NOLOCK) on cd.PatientID = apa.person_id and apa.payer_id = cd.PayerID and apa.end_date >= cd.DateOfService\r\n ) as cd\r\n on cdt.cid = cd.ChargeDetailID", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}, 'dbfsFileName': 'score021', 'directory_path': '/ScoringFiles/DenialsScenario'}")
	#sinkPostExecutionHook(dbfssink)

except Exception as ex: 
	logging.error(ex)
